# Calculating Enrichment scores

Applications that were used: Cytoscape (v.3.8.2) and Jupyter Notebooks (v.2.2.6) through the Anaconda navigator (v.1.10.0).

In Cytoscape, the following app was installed: FileTransfer (v.1.1) for communication to Jupyter notebooks. 
To install apps in Cytoscape go to Apps -> App Manager -> Search

For those new to working with Jupyter Notebooks, go to the [MarkDown Tutorial](https://www.markdowntutorial.com/lesson/1/) to learn how to comment your code properly. 

Now let's move on to the code. 

#### Here we will calculate enrichment scores using a loop. First we walk through making the enrichment score of one KE, then we will make a loop.

### Step 1a: Importing required packages

In [1]:
import os
import time
import sys
import requests
import pandas as pd
import py4cytoscape as p4c
from lxml import etree as ET

## Step 1b: Load code that prevents unwanted logging error

In [2]:
from logging import getLogger, INFO
from concurrent_log_handler import ConcurrentRotatingFileHandler

In [3]:
log = getLogger()
# Use an absolute path to prevent file rotation trouble.
logfile = os.path.abspath("py4cytoscape.log")
# Rotate log after reaching 512K, keep 5 old copies.
rotateHandler = ConcurrentRotatingFileHandler(logfile, "a", 512*1024, 5)
log.addHandler(rotateHandler)
log.setLevel(INFO)

log.info("Here is a very exciting log message, just for you")

Check py4cytoscape version

In [ ]:
p4c.cytoscape_version_info()

## Step 2: Loading an existing network

We are using a previously created network in .cys format. First we will define a new variable to the file path and then a new variable to the path+file name. You can also import files of different formats, such as the .sif format. Look into py4cytoscape documentation on how to import different file types. 

In [5]:
path = 'C:/Users/stefa/Documents/GitHub/2021-internship/Raats/'
file_name = path + 'networkDeleted.cys'
file_name

'C:/Users/stefa/Documents/GitHub/2021-internship/Raats/networkDeleted.cys'

Now we will load this network in cytoscape

In [6]:
abs_file_name = os.path.abspath(file_name)

In [7]:
session_cmd_list = ['session','open','file="',abs_file_name,'"']
session_cmd = " ".join(session_cmd_list)

In [8]:
p4c.commands.commands_run(session_cmd)

[]

## Step 3: Enrichment score for a specific event  
First we will walk through calculating a single KE's enrichment score. This will make it more clear what each step does.  
Afterwards, we create a for loop, which calculates all enrichment scores and puts it into a table. 

## Lists of N,n,B,b

We need to calculate  
N = total number of genes  
n = number of target genes   
B = total number of significant genes  
b = total number of significant target genes

We will calculate this using various filters.

N = column filter type gene   
n = first neighbour of X node  
B = combined filter 1 = column filter type gene & sig < 0.05  
b = combined list = first neighbour of X node & sig < 0.05   

### Filter for all genes

In [32]:
only_genes_filter = p4c.create_column_filter('only_genes_filter','CTL.Type', 'gene',  "IS")

Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


### Filter for significant genes

In [33]:
sig_gene_filter = p4c.create_column_filter('sig_gene_filter','PValue', 0.05, "LESS_THAN")

Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


### Filter for first neighbour of X key event

I chose the key event 'Oxidative stress'.

In [34]:
first_neighbour_list = p4c.get_first_neighbors('Oxidative stress', as_nested_list=False)

### Combine filter for significant p value and genes 

In [35]:
combined_filter_1 = p4c.create_composite_filter('combined_filter_1', ['only_genes_filter', 'sig_gene_filter'])

Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


### Making the filter for significant genes of X key event

In [36]:
dfSigGene = pd.DataFrame(sig_gene_filter)
sig_gene_list = dfSigGene['nodes'].tolist()

sigInKe = list(set(sig_gene_list) & set(first_neighbour_list))

## Step 4: defining the variables
Now we make dataframes of all lists, so we can get the length of all dataframes. Lengths are the numbers we will use for Enrichment calculation

In [57]:
dftotalGenes = pd.DataFrame(only_genes_filter)
dftotalSigGenes = pd.DataFrame(combined_filter_1)
dfFirstNeighbours = pd.DataFrame(first_neighbour_list)
dfSigInKe = pd.DataFrame(sigInKe)
N = float(len(dftotalGenes))
n = float(len(first_neighbour_list))
B = float(len(dftotalSigGenes))
b = float(len(sigInKe))
print(f"\nN: Total number of genes:\n{N}")
print(f"\nn: Total number target set genes:\n{n}")
print(f"\nB: Total number of significant genes:\n{B}")
print(f"\nb: Total number of significant genes in target set:\n{b}")


N: Total number of genes:
210

n: Total number target set genes:
29

B: Total number of significant genes:
66

b: Total number of significant genes in target set:
12


## Step 5: Calculation of enrichment score  
target % = significant genes in KE / total significant genes in set  
background % = significant genes total / total genes in set  
score = target% / background%  

In [15]:
score = (b/B) / (n/N)
score

1.316614420062696

# Step 6: Output table
Before putting steps 3 through 5 in a loop, we create a bit of code to create an output table

In my network, I wish to select all KE. These are coded as 'Pathway'

In [16]:
only_KE_pathways = p4c.create_column_filter("only_KE_pathways", 'Type', 'Pathway', "IS")

Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Now we make a dataframe out of this list. We additionally add and remove some columns. Note that for the EnrichmentScore, we write down 0.0.   
This automatically gives us a variable of the type float. Would we have written 0, it would have selected variable type integer, which we do not want.   

Most importantly, we also make the variable *list_pathways*.

In [21]:
dfEnrichtmentScores = pd.DataFrame(only_KE_pathways)
list_pathways = dfEnrichtmentScores['nodes'].tolist()
list_pathways

['Overview of proinflammatory and profibrotic mediators',
 'ACE2 inhibition',
 'Angiotensin II receptor type 1 pathway',
 'Collagen biosynthesis and modifying enzymes',
 'NF-KB pathway',
 'Oxidative stress',
 'Renin Angiotensin Aldosterone System',
 'Metabolism of Angiotensinogen to Angiotensins']

In [22]:
dfEnrichtmentScores.drop('edges', axis=1, inplace=True)
dfEnrichtmentScores['EnrichmentScore'] = 0.0
dfEnrichtmentScores.set_index('nodes', inplace=True)
dfEnrichtmentScores

,EnrichmentScore
nodes,
Overview of proinflammatory and profibrotic mediators,0.0
ACE2 inhibition,0.0
Angiotensin II receptor type 1 pathway,0.0
Collagen biosynthesis and modifying enzymes,0.0
NF-KB pathway,0.0
Oxidative stress,0.0
Renin Angiotensin Aldosterone System,0.0
Metabolism of Angiotensinogen to Angiotensins,0.0


# Step 7: Loop time!  
We use the list_pathways in our loop, that we have defined earlier.  
It is included in the line that defines *first_neighbour_list*.  
We also used the pathway list in *dfEnrichtmentScores.at*.  

Additionally, we keep track of the time it takes to make the table. For me it takes around 35 seconds.  
  
Note that many cytoscape warnings will pop up, this is okay.   

In [85]:
start_create = time.monotonic()
for pathway in list_pathways: 

    only_genes_filter = p4c.create_column_filter('only_genes_filter','CTL.Type', 'gene',  "IS")
    sig_gene_filter = p4c.create_column_filter('sig_gene_filter','PValue', 0.05, "LESS_THAN")
    first_neighbour_list = p4c.get_first_neighbors(pathway, as_nested_list=False)
    combined_filter_1 = p4c.create_composite_filter('combined_filter_1', ['only_genes_filter', 'sig_gene_filter'])
    dfSigGene = pd.DataFrame(sig_gene_filter)
    sig_gene_list = dfSigGene['nodes'].tolist()

    sigInKe = list(set(sig_gene_list) & set(first_neighbour_list))
    dftotalGenes = pd.DataFrame(only_genes_filter)
    dftotalSigGenes = pd.DataFrame(combined_filter_1)
    dfFirstNeighbours = pd.DataFrame(first_neighbour_list)
    dfSigInKe = pd.DataFrame(sigInKe)
    N = float(len(dftotalGenes))
    n = float(len(first_neighbour_list))
    B = float(len(dftotalSigGenes))
    b = float(len(sigInKe))

    score = float((b/B) / (n/N))
    dfEnrichtmentScores.at[pathway, 'EnrichmentScore'] = score
print(f'create took {(time.monotonic() - start_create):10.2f} seconds')

Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


...

No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.
create took      32.27 seconds


In [24]:
dfEnrichtmentScores

,EnrichmentScore
nodes,
Overview of proinflammatory and profibrotic mediators,1.845455
ACE2 inhibition,0.318182
Angiotensin II receptor type 1 pathway,0.530303
Collagen biosynthesis and modifying enzymes,0.338491
NF-KB pathway,1.060606
Oxidative stress,1.316614
Renin Angiotensin Aldosterone System,0.513196
Metabolism of Angiotensinogen to Angiotensins,0.867769


# Step 8: Loop for significance  

To calculate signifiance for Enrichment scores, we use a hypergeometric distribution formula.   
There might be a package available online that is able to easily import a HGT function. Since it is a small formula, I have decided to code it.   

First we need to import math and create a second column

In [81]:
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom  # or / in Python 2

In [86]:
dfEnrichtmentScores['HGTscore'] = 0.0

Now we calculate the score using the created N,n,B and b. 

In [104]:
start_create = time.monotonic()
for pathway in list_pathways: 

    only_genes_filter = p4c.create_column_filter('only_genes_filter','CTL.Type', 'gene',  "IS")
    sig_gene_filter = p4c.create_column_filter('sig_gene_filter','PValue', 0.05, "LESS_THAN")
    first_neighbour_list = p4c.get_first_neighbors(pathway, as_nested_list=False)
    combined_filter_1 = p4c.create_composite_filter('combined_filter_1', ['only_genes_filter', 'sig_gene_filter'])
    dfSigGene = pd.DataFrame(sig_gene_filter)
    sig_gene_list = dfSigGene['nodes'].tolist()

    sigInKe = list(set(sig_gene_list) & set(first_neighbour_list))
    dftotalGenes = pd.DataFrame(only_genes_filter)
    dftotalSigGenes = pd.DataFrame(combined_filter_1)
    dfFirstNeighbours = pd.DataFrame(first_neighbour_list)
    dfSigInKe = pd.DataFrame(sigInKe)
    N = len(dftotalGenes)
    n = len(first_neighbour_list)
    B = len(dftotalSigGenes)
    b = len(sigInKe)

    num = N-n
    den = B-b
    alpha = ncr(n,b)
    beta = ncr(num, den)
    gamma = ncr(N,B)
    delta = alpha * beta
    
    score = delta / gamma    
    dfEnrichtmentScores.at[pathway, 'HGTscore'] = score
print(f'create took {(time.monotonic() - start_create):10.2f} seconds')

Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


...

No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.


Warning -- Cytoscape version pre-3.9 in use ... settling delay inserted after filter execution


No edges selected.
create took      32.61 seconds


In [105]:
dfEnrichtmentScores

,EnrichmentScore,HGTscore
nodes,,
Overview of proinflammatory and profibrotic mediators,1.845455,0.000005
ACE2 inhibition,0.318182,0.101470
Angiotensin II receptor type 1 pathway,0.530303,0.049078
Collagen biosynthesis and modifying enzymes,0.338491,0.000180
NF-KB pathway,1.060606,0.277184
Oxidative stress,1.316614,0.077468
Renin Angiotensin Aldosterone System,0.513196,0.022770
Metabolism of Angiotensinogen to Angiotensins,0.867769,0.256062


## Step 9: Finishing up
  
Now let us create a third column, which will make it easy to see which KEs are significant and which are not. 

In [114]:
dfEnrichtmentScores['Significant'] = False
dfEnrichtmentScores.loc[dfEnrichtmentScores['HGTscore'] < 0.05, 'Significant'] = True
dfEnrichtmentScores

,EnrichmentScore,HGTscore,Significant
nodes,,,
Overview of proinflammatory and profibrotic mediators,1.845455,0.000005,True
ACE2 inhibition,0.318182,0.101470,False
Angiotensin II receptor type 1 pathway,0.530303,0.049078,True
Collagen biosynthesis and modifying enzymes,0.338491,0.000180,True
NF-KB pathway,1.060606,0.277184,False
Oxidative stress,1.316614,0.077468,False
Renin Angiotensin Aldosterone System,0.513196,0.022770,True
Metabolism of Angiotensinogen to Angiotensins,0.867769,0.256062,False


Finally, let us tidy up the dataframe. 

In [120]:
dfEnrichtmentScores.index.name = 'KE'
dfEnrichtmentScores

,EnrichmentScore,HGTscore,Significant
KE,,,
Overview of proinflammatory and profibrotic mediators,1.845455,0.000005,True
ACE2 inhibition,0.318182,0.101470,False
Angiotensin II receptor type 1 pathway,0.530303,0.049078,True
Collagen biosynthesis and modifying enzymes,0.338491,0.000180,True
NF-KB pathway,1.060606,0.277184,False
Oxidative stress,1.316614,0.077468,False
Renin Angiotensin Aldosterone System,0.513196,0.022770,True
Metabolism of Angiotensinogen to Angiotensins,0.867769,0.256062,False


In [ ]:
dfEnrichmentscore..to_csv(r'Path where you want to store the exported CSV file\File Name.csv', index = False)
